# Homework 2

In [16]:
# imports
import re
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import Binarizer
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

Load the stemmed dataset from Exercise 2 by first mounting the drive and then using
pickle.load(). Print the first entry of the data frame to assure that it was imported
correctly

In [3]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [130]:
#Load stemmed data as .pkl
with open('/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Newsgroup/data_stemmed.pkl', 'rb') as f:
    data_stemmed_list = pickle.load(f)

data_stemmed = pd.DataFrame(data_stemmed_list, columns=["stemmed_text"])

In [5]:
data_stemmed.head(1)

,stemmed_text
0,car wonder enlighten car saw dai door sport ca...


## Bag-of-words (sklearn)
a) Apply the following transformations to the stemmed data using the
fit_transform() function. What is each of them doing? What are the pros and
cons of each approach?
- CountVectorizer(max_df=0.95, min_df=0.05)
- TfidfVectorizer(max_df=0.95, min_df=0.05,use_idf=False, norm='l1')
- TfidfVectorizer(max_df=0.95, min_df=0.05, smooth_idf=False)

In [6]:
abs_vectorizer = CountVectorizer(max_df=0.95, min_df=0.05)
rel_vectorizer = TfidfVectorizer(max_df=0.95, min_df=0.05, use_idf=False, norm='l1') # use-idf: Enable inverse-document-frequency reweighting; norm: Each output row will have unit norm (l1 = Sum of absolute values of vector elements is 1.)
idf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=0.05, smooth_idf=False) # smooth_idf: Smooth idf weights by adding one to document frequencies (Prevents zero divisions) -> not regularized

In [7]:
abs_fre = abs_vectorizer.fit_transform(data_stemmed["stemmed_text"])
abs_fre.toarray()

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
rel_fre = rel_vectorizer.fit_transform(data_stemmed["stemmed_text"])
rel_fre.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.05882353],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01666667, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [9]:
idf_fre = idf_vectorizer.fit_transform(data_stemmed["stemmed_text"])
idf_fre.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.12166272],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.11514646, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

How many features are left in the dictionary in iii. and what are their names
(Hint: Use get_feature_names())? Do the features make sense? Are they
different to those in i. and ii.? What happens if you change max_df and min_df?

In [10]:
len(idf_vectorizer.get_feature_names_out())

246

Apply Binarizer() to the result from a. i. using again fit_transform(). What does
the result stand for? What are its pros and cons?

In [11]:
binar = Binarizer()
abs_bin = binar.fit_transform(abs_fre)
print(abs_fre.toarray().max())
print(abs_bin.toarray().max()) # OHE version (only counts if word occurs, but not how many times)

302
1


c) Put the frequencies for the **first document** from a. and b. in a data frame with a column keys containing the feature names from a. and four additional columns with the corresponding frequencies from a. and b. (Hint: Convert the corpus to
an array first.). Keep only those rows of the data frame with keys representing
words that exist in the first document.

In [51]:
first_doc_df = pd.DataFrame(np.array(np.unique(data_stemmed.iloc[0,0].split())).T, columns=["keys"])
first_doc_df = pd.merge(first_doc_df, pd.DataFrame(abs_fre.toarray(), columns=abs_vectorizer.get_feature_names_out()).iloc[0,:], how="left", left_on="keys", right_index=True)
first_doc_df = pd.merge(first_doc_df, pd.DataFrame(rel_fre.toarray(), columns=rel_vectorizer.get_feature_names_out()).iloc[0,:], how="left", left_on="keys", right_index=True)
first_doc_df = pd.merge(first_doc_df, pd.DataFrame(idf_fre.toarray(), columns=idf_vectorizer.get_feature_names_out()).iloc[0,:], how="left", left_on="keys", right_index=True)
first_doc_df.columns = ["keys", "abs_fre", "rel_fre", "tf_idf_fre"]
first_doc_df = pd.merge(first_doc_df, pd.DataFrame(abs_bin.toarray(), columns=abs_vectorizer.get_feature_names_out()).iloc[0,:], how="left", left_on="keys", right_index=True)
first_doc_df.columns = ["keys", "abs_fre", "rel_fre", "tf_idf_fre", "OHE_fre"]
first_doc_df.fillna(0, inplace=True)
first_doc_df.head()

,keys,abs_fre,rel_fre,tf_idf_fre,OHE_fre
0,addit,0.0,0.0,0.0,0.0
1,bodi,0.0,0.0,0.0,0.0
2,bricklin,0.0,0.0,0.0,0.0
3,brought,0.0,0.0,0.0,0.0
4,bumper,0.0,0.0,0.0,0.0


d) Sort the data frame (descending) on:
- TF-IDF Frequencies
- Absolute Frequencies


Do you see differences? Why?

In [52]:
#tf-idf
first_doc_df.sort_values("tf_idf_fre", ascending=False).head()

,keys,abs_fre,rel_fre,tf_idf_fre,OHE_fre
6,car,5.0,0.294118,0.850894,1.0
18,look,2.0,0.117647,0.236711,1.0
26,small,1.0,0.058824,0.176815,1.0
14,info,1.0,0.058824,0.173306,1.0
10,engin,1.0,0.058824,0.171285,1.0


In [53]:
#abs
first_doc_df.sort_values("abs_fre", ascending=False).head()

,keys,abs_fre,rel_fre,tf_idf_fre,OHE_fre
6,car,5.0,0.294118,0.850894,1.0
18,look,2.0,0.117647,0.236711,1.0
32,year,1.0,0.058824,0.121663,1.0
7,dai,1.0,0.058824,0.140068,1.0
15,know,1.0,0.058824,0.097363,1.0


## Bag-of-words (gensim)

In [65]:
corpus_gen=[doc.split() for doc in data_stemmed_list]

b) Create a gensim Dictionary() based on the corpus. Call it id2word. Apply to it
. What is this method
doing?

In [67]:
id2word = Dictionary(corpus_gen)
id2word.filter_extremes(no_below=566, no_above=0.95)

Apply the following operations. What is each of them doing?
1. print(id2word.token2id)
2. print(id2word.token2id.keys())
3. print(id2word.dfs)
4. corpus1=[id2word.doc2bow(doc) for doc in corpus_gen]
5. corpus2=[[(token[0],(token[1]/sum(n for _, n in doc))) for token in doc] for doc in corpus1]
6. corpus3=[[(token[0],1) for token in doc] for doc in corpus1]
7. tfidf=TfidfModel(dictionary=id2word, normalize=True)

In [71]:
# print token and key
print(id2word.token2id)

{'call': 0, 'car': 1, 'dai': 2, 'engin': 3, 'info': 4, 'know': 5, 'look': 6, 'mail': 7, 'small': 8, 'thank': 9, 'wonder': 10, 'year': 11, 'answer': 12, 'base': 13, 'card': 14, 'edu': 15, 'experi': 16, 'final': 17, 'gui': 18, 'host': 19, 'messag': 20, 'nntp': 21, 'number': 22, 'post': 23, 'report': 24, 'send': 25, 'access': 26, 'actual': 27, 'advanc': 28, 'anybodi': 29, 'better': 30, 'bit': 31, 'email': 32, 'expect': 33, 'feel': 34, 'good': 35, 'got': 36, 'great': 37, 'heard': 38, 'help': 39, 'life': 40, 'like': 41, 'line': 42, 'machin': 43, 'mayb': 44, 'new': 45, 'opinion': 46, 'peopl': 47, 'plai': 48, 'price': 49, 'probabl': 50, 'question': 51, 'read': 52, 'real': 53, 'recent': 54, 'start': 55, 'take': 56, 'time': 57, 'us': 58, 'wai': 59, 'address': 60, 'articl': 61, 'chip': 62, 'com': 63, 'far': 64, 'inform': 65, 'person': 66, 'phone': 67, 'point': 68, 'pretti': 69, 'requir': 70, 'stuff': 71, 'system': 72, 'thing': 73, 'write': 74, 'wrote': 75, 'check': 76, 'mean': 77, 'possibl': 78,

In [72]:
#print keys of dictonary
print(id2word.token2id.keys())

dict_keys(['call', 'car', 'dai', 'engin', 'info', 'know', 'look', 'mail', 'small', 'thank', 'wonder', 'year', 'answer', 'base', 'card', 'edu', 'experi', 'final', 'gui', 'host', 'messag', 'nntp', 'number', 'post', 'report', 'send', 'access', 'actual', 'advanc', 'anybodi', 'better', 'bit', 'email', 'expect', 'feel', 'good', 'got', 'great', 'heard', 'help', 'life', 'like', 'line', 'machin', 'mayb', 'new', 'opinion', 'peopl', 'plai', 'price', 'probabl', 'question', 'read', 'real', 'recent', 'start', 'take', 'time', 'us', 'wai', 'address', 'articl', 'chip', 'com', 'far', 'inform', 'person', 'phone', 'point', 'pretti', 'requir', 'stuff', 'system', 'thing', 'write', 'wrote', 'check', 'mean', 'possibl', 'right', 'set', 'softwar', 'tell', 'understand', 'world', 'ye', 'agre', 'allow', 'apr', 'believ', 'come', 'consid', 'control', 'cost', 'cours', 'exist', 'follow', 'given', 'govern', 'hand', 'hard', 'hope', 'idea', 'john', 'kill', 'make', 'need', 'non', 'power', 'reason', 'result', 'sai', 'secon

In [73]:
# how many documents contain this token
print(id2word.dfs)

{1: 700, 10: 702, 2: 1367, 6: 2215, 0: 1120, 8: 604, 5: 3532, 3: 683, 11: 2058, 4: 653, 7: 1394, 9: 2038, 17: 577, 24: 605, 21: 2335, 23: 3771, 19: 2423, 15: 6270, 22: 1182, 16: 629, 25: 743, 20: 823, 14: 705, 13: 899, 12: 768, 18: 596, 51: 1776, 55: 1234, 40: 750, 59: 2223, 45: 2950, 43: 611, 31: 1008, 44: 809, 29: 568, 33: 582, 38: 728, 26: 640, 49: 617, 42: 997, 41: 3863, 54: 632, 50: 1118, 36: 1223, 34: 710, 30: 1218, 37: 969, 35: 2341, 46: 1074, 47: 2554, 58: 2528, 56: 639, 53: 936, 48: 773, 27: 1099, 39: 1629, 28: 651, 32: 777, 52: 1497, 57: 2835, 63: 3925, 75: 810, 74: 6085, 61: 4995, 62: 581, 64: 961, 71: 574, 69: 654, 70: 735, 68: 1572, 60: 639, 67: 717, 65: 1113, 72: 607, 73: 2074, 66: 1249, 84: 1026, 85: 777, 77: 1487, 83: 733, 81: 829, 76: 652, 79: 1861, 80: 999, 78: 1208, 82: 1214, 112: 871, 102: 958, 88: 2845, 103: 768, 108: 967, 105: 1110, 93: 589, 106: 2188, 92: 930, 98: 848, 110: 688, 107: 750, 95: 831, 113: 1361, 90: 1674, 89: 1436, 100: 855, 114: 970, 86: 662, 99: 78

In [75]:
# Convert document into the bag-of-words (BoW) format = list of (token_id, token_count) tuples. (abs. freq)
corpus1=[id2word.doc2bow(doc) for doc in corpus_gen]
corpus1[0]

[(0, 1),
 (1, 5),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1)]

In [76]:
# Convert document into the bag-of-words (BoW) format = list of (token_id, token_count) tuples. (rel. freq)
corpus2=[[(token[0],(token[1]/sum(n for _, n in doc))) for token in doc] for doc in corpus1]
corpus2[0]

[(0, 0.058823529411764705),
 (1, 0.29411764705882354),
 (2, 0.058823529411764705),
 (3, 0.058823529411764705),
 (4, 0.058823529411764705),
 (5, 0.058823529411764705),
 (6, 0.11764705882352941),
 (7, 0.058823529411764705),
 (8, 0.058823529411764705),
 (9, 0.058823529411764705),
 (10, 0.058823529411764705),
 (11, 0.058823529411764705)]

In [77]:
# OHE 
corpus3=[[(token[0],1) for token in doc] for doc in corpus1]
corpus3[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1)]

In [91]:
# tf-idf 
tfidf=TfidfModel(dictionary=id2word, normalize=True)
tfidf[corpus1[0]]

[(0, 0.1439962983999712),
 (1, 0.8663005681608876),
 (2, 0.13158792186479112),
 (3, 0.17479087800687215),
 (4, 0.1775875886665189),
 (5, 0.07248507716774467),
 (6, 0.20307542403853562),
 (7, 0.13037013599023706),
 (8, 0.1824442817393251),
 (9, 0.10672318695643962),
 (10, 0.17308247312399705),
 (11, 0.10611514544559189)]

Put all the frequencies from iv. to vii. in a data frame for the first document in
the corpus. Add a column keys with the feature names. Compare the results
with those from sklearn by merging the two data frames. Where do you think
that the differences come from?

In [99]:
firstdoc_gen_df = pd.DataFrame(list(id2word.token2id.keys())[:len(corpus1[0])], columns=["keys"])
firstdoc_gen_df

,keys
0,call
1,car
2,dai
3,engin
4,info
5,know
6,look
7,mail
8,small
9,thank


In [121]:
firstdoc_gen_df = pd.DataFrame(np.hstack((corpus1[0], corpus2[0], corpus3[0], tfidf[corpus1[0]]))[:,1::2], columns=["abs_fre", "rel_fre", "OHE_fre", "tf_idf_fre"])
firstdoc_gen_df["keys"] = list(id2word.token2id.keys())[:len(corpus1[0])]
firstdoc_gen_df

,abs_fre,rel_fre,OHE_fre,tf_idf_fre,keys
0,1.0,0.058824,1.0,0.143996,call
1,5.0,0.294118,1.0,0.866301,car
2,1.0,0.058824,1.0,0.131588,dai
3,1.0,0.058824,1.0,0.174791,engin
4,1.0,0.058824,1.0,0.177588,info
5,1.0,0.058824,1.0,0.072485,know
6,2.0,0.117647,1.0,0.203075,look
7,1.0,0.058824,1.0,0.130370,mail
8,1.0,0.058824,1.0,0.182444,small
9,1.0,0.058824,1.0,0.106723,thank


In [122]:
first_doc_df

,keys,abs_fre,rel_fre,tf_idf_fre,OHE_fre
0,addit,0.0,0.000000,0.000000,0.0
1,bodi,0.0,0.000000,0.000000,0.0
2,bricklin,0.0,0.000000,0.000000,0.0
3,brought,0.0,0.000000,0.000000,0.0
4,bumper,0.0,0.000000,0.000000,0.0
5,call,1.0,0.058824,0.149034,1.0
6,car,5.0,0.294118,0.850894,1.0
7,dai,1.0,0.058824,0.140068,1.0
8,door,0.0,0.000000,0.000000,0.0
9,earli,0.0,0.000000,0.000000,0.0


The main difference between sklearn and gensim is that during the creation of the dictonary we set upper and lower bounds, which reduced the number of words in the dictonary

## Ngrams

a) Apply the following transformation to the stemmed data using fit_transform():
CountVectorizer(ngram_range=(2, 2), max_df=0.95, min_df=0.05). What is it
doing? Print the corresponding features. What are the differences to the
features in Question 3. and Question 4? What happens if you change the values
of max_df and min_df?


In [142]:
ngrams_vec = CountVectorizer(ngram_range=(2, 2), max_df=0.95, min_df=0.05)

bigrams = ngrams_vec.fit_transform(data_stemmed["stemmed_text"])

In [145]:
feature_names = ngrams_vec.get_feature_names_out()

b) Put the result in a data frame with columns=feature_names. Print the data frame
head and the maximum values for each word. What do you notice?

In [148]:
bigram_df = pd.DataFrame(bigrams.toarray(), columns=feature_names)
bigram_df.head()

,articl apr,edu write,nntp post,post host,write articl
0,0,0,0,0,0
1,0,0,1,1,0
2,0,0,0,0,0
3,0,1,1,1,1
4,0,0,0,0,0
